In [ ]:
import os
import requests
import base64
import os
import openai
from openai import OpenAI
import pandas as pd
import numpy as np
import json
import glob
from collections import defaultdict
import time
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

Fetching info from clinicaltrial.gov using API

In [ ]:
!mkdir -p '/content/drive/MyDrive/ChatNCCN/cancer_icfs_21to24'
pattern = r"https://\S*/ICF_0\d\d+\.pdf"

for i in cancer_df_21to24.iterrows():
  nct_id = i[1]['NCT Number']
  urls = urls = re.findall(pattern, i[1]['Study Documents'])
  if len(urls) > 1 and (nct_id == 'NCT04789720' or nct_id == 'NCT06183788' or nct_id == 'NCT04975698'):
    urls = [urls[0]]
  print(urls)
  for j, url in enumerate(urls):
    r = requests.get(url)
    if len(urls) > 1:
      with open(f'/content/drive/MyDrive/ChatNCCN/cancer_icfs_21to24/{nct_id}_{j}.pdf', 'wb') as f:
        f.write(r.content)
    else:
      with open(f'/content/drive/MyDrive/ChatNCCN/cancer_icfs_21to24/{nct_id}.pdf', 'wb') as f:
        f.write(r.content)

Import expert-created MCQAs examples

In [ ]:
example_multiple_choices_questions = pd.read_excel('/content/drive/MyDrive/ChatNCCN/Example MCQA test for NCT03923790.xlsx')
example_multiple_choices_questions

Processing the given examples

In [ ]:
questions = example_multiple_choices_questions['Question'][example_multiple_choices_questions['Question'] != 'N/A (does not appear in consent)']
options = example_multiple_choices_questions['Multi-Choice Options'].dropna()
topics = example_multiple_choices_questions['Topic']
print(len(questions), len(options), len(topics))

topics_notna = topics.dropna()
topics_notna_index = set(topics_notna.index)
i = 0
while i < len(topics):
  if i in topics_notna_index:
    current_topic = topics.iloc[i].strip()
    i += 1
    while i < len(topics) and i not in topics_notna_index:
      topics.iloc[i] = current_topic
      i += 1
topics = topics.iloc[questions.index]
topics

In [ ]:
!mkdir -p '/content/drive/MyDrive/ChatNCCN/nct_mcqa_cancer_icfs'
out_dir = '/content/drive/MyDrive/ChatNCCN/nct_mcqa_cancer_icfs'
in_dir = '/content/drive/MyDrive/ChatNCCN'

# importing required modules
from PyPDF2 import PdfReader

def convert_pdf_to_text(pdf_path):
  # creating a pdf reader object
  reader = PdfReader(pdf_path)

  # printing number of pages in pdf file
  print(len(reader.pages))

  example_consent_form = ''
  for i in range(len(reader.pages)):
    page_text = reader.pages[i].extract_text()
    if page_text == '':
      return ''
    example_consent_form += page_text.strip()
  return example_consent_form

Get text of the example ICF (used for creating exmaple MCQAs)

In [ ]:
example_consent_form = convert_pdf_to_text(os.path.join(in_dir, 'example_consent_form_nct03923790.pdf'))
example_consent_form

Select NIH topics that show up in the example ICF

In [ ]:
topics_notna_index_set = set(list(topics_notna_index))
questions_index_set = set(list(questions.index))
intersect_set = topics_notna_index_set.intersection(questions_index_set)
interse_set = intersect_set.intersection(set(questions))
print(intersect_set)

In [ ]:
raw_topics = example_multiple_choices_questions['Topic']
raw_questions = example_multiple_choices_questions['Question']
raw_options = example_multiple_choices_questions['Multi-Choice Options']
raw_correct_answers = example_multiple_choices_questions['Correct Answer']

Generating MCQAs with in-context example

In [ ]:
def get_chat_completion_mcqa(user_prompt1, user_prompt2, assistant_prompt, system_prompt,
                             engine='gpt-4-1106-preview',
                             temperature=0, max_tokens=3000, top_p=0,
                             frequency_penalty=0, presence_penalty=0, stop=None):
    """
    Generates a chat completion using OpenAI's GPT model.

    Parameters:
    - user_prompt (str): The user's input prompt to the model.
    - system_prompt (str): The system's initial prompt setting the context for the model.
    - engine (str): The model you are using: [gpt-4, gpt4-32k, gpt-35-turbo-0613]
    - temperature (float): Controls randomness in the generation.
    - max_tokens (int): The maximum number of tokens to generate in the completion.
    - top_p (float): Nucleus sampling parameter controlling the size of the probability mass considered for token generation.
    - frequency_penalty (float): How much to penalize new tokens based on their frequency.
    - presence_penalty (float): How much to penalize new tokens based on their presence.
    - stop (list or None): Tokens at which to stop generating further tokens.

    Returns:
    - str: The generated completion text.
    """
    client = OpenAI(api_key=GPT4V_KEY)

    message_text = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt1},
        {"role": "assistant", "content": assistant_prompt},
        {"role": "user", "content": user_prompt2}
    ]

    completion = client.chat.completions.create(
        model=engine,
        messages=message_text,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop,
        # logprobs=True,
        # top_logprobs=5
    ).choices[0].message.content

    return completion

In [ ]:
system_prompt = 'You are a smart AI assistant. Given an example consent form and a multiple-choices question in a specific topic regarding the form that helps patient understand the form. Now generate new questions regarding the new form in the same topic.'
pdf_list = pd.read_csv('/content/drive/MyDrive/ChatNCCN/cancer_icf_ids')
for i, nct in enumerate(pdf_list):
  pdf = os.path.join('/content/drive/MyDrive/ChatNCCN/cancer_icfs_21to24', f'{nct}.pdf')
  nct_text = convert_pdf_to_text(pdf)
  print(f'Processing {nct}........')
  if nct_text == '':
    print('Got empty text.........')
    continue
  nct_mcqa_dict_with_example = defaultdict(dict)
  for t in intersect_set:
    if t == 46:
      continue
    topic = raw_topics.iloc[t]
    print(f'Processing {topic}........')
    example_question_options = raw_questions.iloc[t] + '\n ' + raw_options.iloc[t] + '\n Correct answer: ' + raw_correct_answers.iloc[t]
    user_prompt1 = f'===Example consent form===: \n {example_consent_form} \n\n ===Topic===: \n {topic} \n Generate a multiple-choices question in the given topic regarding the given consent form.'
    assistant_prompt = f'===Example question===: \n {example_question_options}'
    user_prompt2 = f'===New consent form===: \n {nct_text} \n\n ===Topic===: \n {topic} \n Generate one multiple-choices question in the given topic regarding the new consent form; the correct option of each new question should not be the original sentences from the consent form.'
    nct_mcqa_dict_with_example[topic] = get_chat_completion_mcqa(user_prompt1, user_prompt2, assistant_prompt, system_prompt).replace('===New question===:\n', '')
  topics = nct_mcqa_dict_with_example.keys()
  questions = nct_mcqa_dict_with_example.values()
  df = pd.DataFrame({'Topics': list(topics), 'Questions': questions})
  df.to_csv(os.path.join(out_dir, f'{nct}_mcqa_gpt4_turbo_with_example.csv'))